<a href="https://colab.research.google.com/github/arjunparmar/TUNEX/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import PIL
import pathlib
import tensorflow as tf

In [35]:
data_dir = "/content/drive/MyDrive/dataset/main"
data_dir = pathlib.Path(data_dir)
img_height = 180
img_width = 180
batch_size = 32

In [36]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=5,
  shuffle=True,
  subset="training",
  image_size=(img_height, img_width),
  batch_size=32)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=5,
  shuffle=True,
  subset="validation",
  image_size=(img_height, img_width),
  batch_size=32)

Found 17676 files belonging to 7 classes.
Using 14141 files for training.
Found 17676 files belonging to 7 classes.
Using 3535 files for validation.


In [4]:
classes = train_ds.class_names
print(classes)

['0', '1', '2', '3', '4', '5', '6']


In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [43]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', classes=7,
    input_shape=(180, 180, 3),
    classifier_activation='softmax'
)

In [46]:
model = vgg16
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 180, 180, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 180, 180, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 90, 90, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 90, 90, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 90, 90, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 45, 45, 128)       0     

In [39]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>0.90):
        print("\n 90% val acc reached")
        self.model.stop_training = True

In [42]:
model.fit(
  train,
  validation_data=validation,
  epochs=15,
  callbacks=[CustomCallbacks()]
)

Epoch 1/15


InvalidArgumentError: ignored